In [15]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import os

Cell_phones = pd.read_json(r'C:\Users\babin\Downloads\archive (7)\Toys_and_Games\Cell_phones_and_Accessories.json', lines=True)

In [17]:
Cell_phones.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1997140 entries, 0 to 1997139
Data columns (total 12 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   _id             object
 1   reviewerID      object
 2   asin            object
 3   reviewerName    object
 4   helpful         object
 5   reviewText      object
 6   overall         int64 
 7   summary         object
 8   unixReviewTime  int64 
 9   reviewTime      object
 10  category        object
 11  class           int64 
dtypes: int64(3), object(9)
memory usage: 182.8+ MB


In [18]:
Cell_phones['unixReviewTime'] = pd.to_datetime(Cell_phones['unixReviewTime'],unit='s')

In [19]:
Cell_phones= Cell_phones.rename(columns={'overall': 'Product_rating'})
Cell_phones= Cell_phones.rename(columns={'helpful': 'Review_rating'}) 
Cell_phones= Cell_phones.rename(columns={'reviewerName': 'Reviewer_name'})
Cell_phones= Cell_phones.rename(columns={'asin': 'Product_Id'})
Cell_phones= Cell_phones.rename(columns={'unixReviewTime': 'Time_of_review'}) 

In [20]:
Cell_phones.drop('reviewTime',axis=1,inplace=True)
Cell_phones.drop('_id',axis=1,inplace=True)
Cell_phones.drop('reviewerID',axis=1,inplace=True)

In [22]:
null_count = Cell_phones.isnull().sum()
print(null_count)

Product_Id            0
Reviewer_name     10782
Review_rating         0
reviewText            0
Product_rating        0
summary               0
Time_of_review        0
category              0
class                 0
dtype: int64


In [24]:
Cell_phones.value_counts('class')

class
1    1662754
0     334386
dtype: int64

In [25]:
Cell_phones['reviewText'] = Cell_phones['reviewText'].str.lower()

In [27]:
Cell_phones.to_csv(r'C:\Users\babin\OneDrive\Desktop\review\reviewsystem\static/Cell_phones_and_Accessories.csv', index=False)

In [32]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Load the dataset
csv_file_path = r'C:\Users\babin\OneDrive\Desktop\review\reviewsystem\static/Cell_Phones_and_Accessories.csv'
df = pd.read_csv(csv_file_path)

df.head()

df = df[['reviewText','class']]
df.head()

,reviewText,class
0,i love these felt nursery rhyme characters and...,1
1,i see no directions for its use. therefore i h...,0
2,this is a great tool for any teacher using the...,1
3,"great product, thank you! our son loved the pu...",1
4,although not as streamlined as the algebra i m...,1


In [35]:
df2 = df.sample(2000,random_state=42)
df2.rename(columns={'reviewText':'text','class':'label'},inplace=True)
df2 = df2[df2['text'].str.len()<=512]

In [36]:
df2['label'].value_counts(), df2.shape

(1    1395
 0     266
 Name: label, dtype: int64,
 (1661, 2))

In [37]:
df.shape

(1997140, 2)

In [53]:
reviews = df2['text'].tolist()
labels = df2['label'].tolist()

# Text cleaning function
def clean_text(text):
    import re
    text = re.sub(r'\s+', ' ', text)  
    text = text.strip()  
    text = text.lower()  
    return text

# Clean the reviews
reviews = [clean_text(review) for review in reviews]

# Create TFIDF vectorizer and fit it on the data
vectorizer = TfidfVectorizer(max_features=5000)  
X = vectorizer.fit_transform(reviews)  

# Save 
vectorizer_path = r'C:\Users\babin\OneDrive\Desktop\review\reviewsystem\static\TFIDF_vectorization_v2.pkl'
joblib.dump(vectorizer, vectorizer_path)

print(f"TF-IDF vectorizer saved to {vectorizer_path}")


TF-IDF vectorizer saved to C:\Users\babin\OneDrive\Desktop\review\reviewsystem\static\TFIDF_vectorization_v2.pkl


In [46]:
# Define Dataset class
class ReviewDataset(Dataset):
    def __init__(self, reviews, labels, vectorizer):
        self.reviews = reviews
        self.labels = labels
        self.vectorizer = vectorizer
    
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, idx):
        text = self.reviews[idx]
        text_vectorized = self.vectorizer.transform([text]).toarray()[0]
        label = self.labels[idx]
        return torch.tensor(text_vectorized, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# Split the data
reviews_train, reviews_val, labels_train, labels_val = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Create Dataset and DataLoader
train_dataset = ReviewDataset(reviews_train, labels_train, vectorizer)
val_dataset = ReviewDataset(reviews_val, labels_val, vectorizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [47]:
# Define the LSTM model (without embedding layer)
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, text):
        lstm_out, (hidden, cell) = self.lstm(text)
        return torch.sigmoid(self.fc(self.dropout(hidden[-1])))

# Initialize model
input_dim = len(vectorizer.get_feature_names_out())  
hidden_dim = 256  
output_dim = 1  

model = LSTMModel(input_dim, hidden_dim, output_dim)




In [48]:
# Training setup
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [49]:
# Training loop
num_epochs = 10  
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for texts, labels in train_loader:
        optimizer.zero_grad()
        texts = texts.unsqueeze(1)  
        predictions = model(texts).squeeze(1)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader)}')


Epoch 1/10, Loss: 0.5937299025910241
Epoch 2/10, Loss: 0.4320789554289409
Epoch 3/10, Loss: 0.38276169839359464
Epoch 4/10, Loss: 0.3395647055336407
Epoch 5/10, Loss: 0.28845942162332083
Epoch 6/10, Loss: 0.21650937164113634
Epoch 7/10, Loss: 0.14342074681605613
Epoch 8/10, Loss: 0.09527021859373365
Epoch 9/10, Loss: 0.06373547558628377
Epoch 10/10, Loss: 0.04502578629624276


In [50]:
# Validation (optional)
model.eval()
val_loss = 0
with torch.no_grad():
    for texts, labels in val_loader:
        texts = texts.unsqueeze(1)  
        predictions = model(texts).squeeze(1)
        loss = criterion(predictions, labels)
        val_loss += loss.item()

    print(f'Validation Loss: {val_loss/len(val_loader)}')

Validation Loss: 0.2808732424270023


In [51]:
# Save the trained model state
torch.save(model.state_dict(),r'C:\Users\babin\OneDrive\Desktop\review\reviewsystem\static\LSTM_model_state.pth')